In [8]:
from glob import glob

import matplotlib.pyplot as plt  
import matplotlib as mpl  
import numpy as np  
import openslide  
from openslide import open_slide  
from openslide.deepzoom import DeepZoomGenerator  
import pandas as pd

from scipy.ndimage.morphology import binary_fill_holes, binary_closing, binary_dilation  

plt.rcParams['figure.figsize'] = (10, 6)  

In [3]:
#Get .svs images from slides folder
data_type = "validation" # change to train or validation
slide_type = "lung" # change to lung or liver

read_slides_path = "slides/" + data_type + "/" + slide_type + "s/"
write_slides_path = "data/" + data_type + "/" + slide_type + "s/"

slides = glob(read_slides_path + "*.svs")  
print("Slides: " + str(slides) + "\n")

num_slides = len(slides)
print("num_slides: " + str(num_slides))

Slides: ['slides/validation/lungs/GTEX-111VG-0726.svs', 'slides/validation/lungs/GTEX-1122O-0126.svs']

num_slides: 2


In [ ]:
tile_size = 128
default_slide_level = 12 # set to 12 by default

In [ ]:
def not_mostly_white(tile):
    ''' 
    Returns true if more than X% of slide pixels are not white
    '''
    threshold_x = 0.5
    pixels = tile.getdata()
    white_thresh = (240,240,240)
    not_white = 0
    for pixel in pixels:        
        if pixel < white_thresh:
            not_white += 1
    return (not_white /len(pixels)) > threshold_x

In [ ]:
num_saved = 0

for slide_num in range(num_slides):
    print("######### Opening Slide:" + slides[slide_num] + "\n")
    slide = open_slide(slides[slide_num])  
    
    
    tiles = DeepZoomGenerator(slide, tile_size=tile_size, overlap=0, limit_bounds=False)  
    # overlap: adds pixels to each side
    
    print("Level tiles for default level: " + str(tiles.level_tiles[default_slide_level])) 
    # See how many tiles there are for default slide level of magnification.
    
    tiles_x = tiles.level_tiles[default_slide_level][0]
    tiles_y = tiles.level_tiles[default_slide_level][1] 
    
    # iterate through slide level rows and columns
    for i in range(tiles_x):
        for j in range(tiles_y):
            tile = tiles.get_tile(default_slide_level, (i,j))
            if tile.size == (tile_size, tile_size) and not_mostly_white(tile):
                num_saved += 1
                tile_name = write_slides_path + slide_type + str(num_saved) + ".png"
                tile.save(tile_name, "PNG")
                print(tile_name + " saved!")
                
    print("######### Closing Slide:" + slides[slide_num] + "\n")
    slide.close()